In [ ]:
!pip install tensorflow-gpu==2.6.2
!pip install Pillow
!pip install scipy

In [2]:
#from google.colab import drive
#drive.mount('/content/gdrive')

import numpy as np
import tensorflow as tf
print(tf.__version__)
import pandas as pd
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, MaxPool2D, Reshape
from tensorflow.keras.layers import LSTM

2.7.0


In [3]:
# https://stackoverflow.com/questions/58352326/running-the-tensorflow-2-0-code-gives-valueerror-tf-function-decorated-functio
tf.config.run_functions_eagerly(True)

In [3]:
acc = pd.read_csv('data/Activity recognition exp/Phones_accelerometer.csv')
gyro = pd.read_csv('data/Activity recognition exp/Phones_gyroscope.csv')

In [4]:
acc.rename(columns={'x': 'acc_x', 'y': 'acc_y','z': 'acc_z'}, inplace=True)
acc = acc.drop('Index', 1)
acc

/tmp/ipykernel_176266/3864482491.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  acc = acc.drop('Index', 1)


,Arrival_Time,Creation_Time,acc_x,acc_y,acc_z,User,Model,Device,gt
0,1424696633908,1424696631913248572,-5.958191,0.688065,8.135345,a,nexus4,nexus4_1,stand
1,1424696633909,1424696631918283972,-5.952240,0.670212,8.136536,a,nexus4,nexus4_1,stand
2,1424696633918,1424696631923288855,-5.995087,0.653549,8.204376,a,nexus4,nexus4_1,stand
3,1424696633919,1424696631928385290,-5.942718,0.676163,8.128204,a,nexus4,nexus4_1,stand
4,1424696633929,1424696631933420691,-5.991516,0.641647,8.135345,a,nexus4,nexus4_1,stand
...,...,...,...,...,...,...,...,...,...
13062470,1424778553315,92263781761000,1.379043,0.153227,9.959755,i,samsungold,samsungold_2,bike
13062471,1424778553346,92263812248000,1.379043,0.153227,9.806528,i,samsungold,samsungold_2,bike
13062472,1424778553366,92263832267000,1.532270,0.153227,9.806528,i,samsungold,samsungold_2,bike
13062473,1424778553386,92263852409000,1.532270,0.000000,9.959755,i,samsungold,samsungold_2,bike


In [5]:
gyro.rename(columns={'x': 'gyro_x', 'y': 'gyro_y','z': 'gyro_z'}, inplace=True)
gyro = gyro.drop('Index', 1)
gyro

/tmp/ipykernel_176266/574328431.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  gyro = gyro.drop('Index', 1)


,Arrival_Time,Creation_Time,gyro_x,gyro_y,gyro_z,User,Model,Device,gt
0,1424696633909,1424696631914042029,0.013748,-0.000626,-0.023376,a,nexus4,nexus4_1,stand
1,1424696633909,1424696631919046912,0.014816,-0.001694,-0.022308,a,nexus4,nexus4_1,stand
2,1424696633918,1424696631924051794,0.015884,-0.001694,-0.021240,a,nexus4,nexus4_1,stand
3,1424696633919,1424696631929117712,0.016953,-0.003830,-0.020172,a,nexus4,nexus4_1,stand
4,1424696633928,1424696631934214148,0.015884,-0.007034,-0.020172,a,nexus4,nexus4_1,stand
...,...,...,...,...,...,...,...,...,...
13932627,1424778269892,79199637493000,-0.046844,0.337667,0.134677,i,s3mini,s3mini_2,bike
13932628,1424778269906,79199651618000,-0.117598,0.221777,0.131749,i,s3mini,s3mini_2,bike
13932629,1424778269920,79199665927000,-0.177617,0.056115,0.095152,i,s3mini,s3mini_2,bike
13932630,1424778269932,79199677489000,-0.195183,-0.124429,0.063191,i,s3mini,s3mini_2,bike


In [6]:
df = acc.merge(gyro, on='Arrival_Time')
df

,Arrival_Time,Creation_Time_x,acc_x,acc_y,acc_z,User_x,Model_x,Device_x,gt_x,Creation_Time_y,gyro_x,gyro_y,gyro_z,User_y,Model_y,Device_y,gt_y
0,1424696633908,1424696631913248572,-5.958191,0.688065,8.135345,a,nexus4,nexus4_1,stand,274248791573000,0.024129,-0.020769,0.008552,a,s3,s3_2,stand
1,1424696633909,1424696631918283972,-5.952240,0.670212,8.136536,a,nexus4,nexus4_1,stand,1424696631914042029,0.013748,-0.000626,-0.023376,a,nexus4,nexus4_1,stand
2,1424696633909,1424696631918283972,-5.952240,0.670212,8.136536,a,nexus4,nexus4_1,stand,1424696631919046912,0.014816,-0.001694,-0.022308,a,nexus4,nexus4_1,stand
3,1424696633918,1424696631923288855,-5.995087,0.653549,8.204376,a,nexus4,nexus4_1,stand,1424696631924051794,0.015884,-0.001694,-0.021240,a,nexus4,nexus4_1,stand
4,1424696633918,1424696631923288855,-5.995087,0.653549,8.204376,a,nexus4,nexus4_1,stand,274248801573000,0.023518,-0.017410,0.007636,a,s3,s3_2,stand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12726170,1424778553235,92263701744000,1.532270,0.000000,9.806528,i,samsungold,samsungold_2,bike,1424778556726097080,-0.017624,0.005386,0.008514,i,nexus4,nexus4_1,bike
12726171,1424778553256,92263722252000,1.379043,0.000000,9.959755,i,samsungold,samsungold_2,bike,2902562935000,-0.032693,-0.012687,-0.008539,i,s3mini,s3mini_1,bike
12726172,1424778553286,92263752250000,1.532270,0.153227,9.959755,i,samsungold,samsungold_2,bike,2902592809000,-0.031717,-0.006831,-0.006099,i,s3mini,s3mini_1,bike
12726173,1424778553306,92263772270000,1.379043,0.000000,9.959755,i,samsungold,samsungold_2,bike,2902612796000,-0.030741,-0.002928,-0.003904,i,s3mini,s3mini_1,bike


In [7]:
df.rename(columns={'Creation_Time_x': 'Creation_Time_acc', 'Creation_Time_y': 'Creation_Time_gyro', 'User_x': 'User_acc', 'User_y': 'User_gyro', 'Device_x': 'Device_acc', 'Device_y': 'Device_gyro', 'gt_x': 'gt_acc', 'gt_y': 'gt_gyro', 'Model_x': 'Model_acc', 'Model_y': 'Model_gyro'}, inplace=True)

In [8]:
df

,Arrival_Time,Creation_Time_acc,acc_x,acc_y,acc_z,User_acc,Model_acc,Device_acc,gt_acc,Creation_Time_gyro,gyro_x,gyro_y,gyro_z,User_gyro,Model_gyro,Device_gyro,gt_gyro
0,1424696633908,1424696631913248572,-5.958191,0.688065,8.135345,a,nexus4,nexus4_1,stand,274248791573000,0.024129,-0.020769,0.008552,a,s3,s3_2,stand
1,1424696633909,1424696631918283972,-5.952240,0.670212,8.136536,a,nexus4,nexus4_1,stand,1424696631914042029,0.013748,-0.000626,-0.023376,a,nexus4,nexus4_1,stand
2,1424696633909,1424696631918283972,-5.952240,0.670212,8.136536,a,nexus4,nexus4_1,stand,1424696631919046912,0.014816,-0.001694,-0.022308,a,nexus4,nexus4_1,stand
3,1424696633918,1424696631923288855,-5.995087,0.653549,8.204376,a,nexus4,nexus4_1,stand,1424696631924051794,0.015884,-0.001694,-0.021240,a,nexus4,nexus4_1,stand
4,1424696633918,1424696631923288855,-5.995087,0.653549,8.204376,a,nexus4,nexus4_1,stand,274248801573000,0.023518,-0.017410,0.007636,a,s3,s3_2,stand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12726170,1424778553235,92263701744000,1.532270,0.000000,9.806528,i,samsungold,samsungold_2,bike,1424778556726097080,-0.017624,0.005386,0.008514,i,nexus4,nexus4_1,bike
12726171,1424778553256,92263722252000,1.379043,0.000000,9.959755,i,samsungold,samsungold_2,bike,2902562935000,-0.032693,-0.012687,-0.008539,i,s3mini,s3mini_1,bike
12726172,1424778553286,92263752250000,1.532270,0.153227,9.959755,i,samsungold,samsungold_2,bike,2902592809000,-0.031717,-0.006831,-0.006099,i,s3mini,s3mini_1,bike
12726173,1424778553306,92263772270000,1.379043,0.000000,9.959755,i,samsungold,samsungold_2,bike,2902612796000,-0.030741,-0.002928,-0.003904,i,s3mini,s3mini_1,bike


In [9]:
df.to_csv('data/phones.csv')

In [ ]:
phone = pd.read_csv('data/phones.csv')

/home/ale/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (9,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
phone = phone.loc[phone['User_acc'] == 'a']

In [5]:
phone = phone.drop('Arrival_Time', 1)
phone = phone.drop('Creation_Time_acc', 1)
phone = phone.drop('User_acc', 1)
phone = phone.drop('Model_acc', 1)
phone = phone.drop('Device_acc', 1)
phone = phone.drop('gt_acc', 1)
phone = phone.drop('Creation_Time_gyro', 1)
phone = phone.drop('User_gyro', 1)
phone = phone.drop('Model_gyro', 1)
phone = phone.drop('Device_gyro', 1)

/tmp/ipykernel_186345/172377305.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  phone = phone.drop('Arrival_Time', 1)
/tmp/ipykernel_186345/172377305.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  phone = phone.drop('Creation_Time_acc', 1)
/tmp/ipykernel_186345/172377305.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  phone = phone.drop('User_acc', 1)
/tmp/ipykernel_186345/172377305.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  phone = phone.drop('Model_acc', 1)
/tmp/ipykernel_186345/172377305.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

In [6]:
phone.rename(columns={'gt_gyro': 'label'}, inplace=True)
phone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1395809 entries, 0 to 1395808
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1395809 non-null  int64  
 1   acc_x       1395809 non-null  float64
 2   acc_y       1395809 non-null  float64
 3   acc_z       1395809 non-null  float64
 4   gyro_x      1395809 non-null  float64
 5   gyro_y      1395809 non-null  float64
 6   gyro_z      1395809 non-null  float64
 7   label       1255717 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 95.8+ MB


In [7]:
phone = phone.dropna()
phone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1255717 entries, 0 to 1395808
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1255717 non-null  int64  
 1   acc_x       1255717 non-null  float64
 2   acc_y       1255717 non-null  float64
 3   acc_z       1255717 non-null  float64
 4   gyro_x      1255717 non-null  float64
 5   gyro_y      1255717 non-null  float64
 6   gyro_z      1255717 non-null  float64
 7   label       1255717 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 86.2+ MB


In [8]:
phone.to_csv('data/phones_user_a.csv')

In [1]:
X = phone[['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']]

NameError: name 'phone' is not defined

In [10]:
y = phone[['label']]

In [11]:
#from sklearn.preprocessing import LabelEncoder

#def encode_df(dataframe):
#    le = LabelEncoder()
#    for column in dataframe.columns:
#        dataframe[column] = le.fit_transform(dataframe[column])
#    return dataframe

#encode the dataframe
#encode_df(y)
#y

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [32]:
X_train.to_csv('data/X_train.csv')
X_train.shape

(1004573, 6)

In [33]:
X_test.to_csv('data/X_test.csv')
np.save('data/X_test.npy', X_test)
X_test.shape

(251144, 6)

In [31]:
y_train = pd.read_csv('data/y_train.csv')
y_train = y_train.drop('Unnamed: 0', 1)
y_train

/tmp/ipykernel_204972/2431242979.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  y_train = y_train.drop('Unnamed: 0', 1)


,label
0,stairsup
1,stand
2,stairsdown
3,walk
4,stairsdown
...,...
1004568,stand
1004569,bike
1004570,walk
1004571,bike


In [32]:
n_rows = y_train.shape[0]
n_cols = y_train.shape[1]
seg_shape = int(n_rows/400)
seg_shape = seg_shape * 400
seg_shape

1004400

In [33]:
y_train = y_train.head(seg_shape)
y_train

,label
0,stairsup
1,stand
2,stairsdown
3,walk
4,stairsdown
...,...
1004395,walk
1004396,walk
1004397,walk
1004398,bike


In [40]:
# one hot encoding by pandas
y_train = pd.get_dummies(data=y_train, columns=['label'])
y_train.shape

(1004400, 6)

In [41]:
y_train.to_csv('data/y_train.csv')

In [62]:
labels = y_train.to_numpy()
labels.shape

(1004400, 6)

In [63]:
n_instances = labels.shape[0]
n_attributes = labels.shape[1]
labels = labels.reshape(n_instances, n_attributes)
np.save('data/y_train.npy', labels) 
labels.shape

(1004400, 6)

In [51]:
np.save('data/y_train.npy', labels)

In [35]:
y_test.to_csv('data/y_test.cvs')
np.save('data/y_test.npy', y_test)
y_test.shape

(251144, 1)

In [52]:
X_train = pd.read_csv('data/X_train.csv')
X_train = X_train.drop('Unnamed: 0', 1)
X_train

/tmp/ipykernel_204972/561670757.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = X_train.drop('Unnamed: 0', 1)


,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z
0,-6.147430,1.798508,7.260559,0.852661,-0.562042,-0.006714
1,-5.669399,-0.153227,7.814577,-0.040317,0.056505,0.080634
2,-4.187195,-1.955337,7.799713,0.203247,0.073486,0.088348
3,-5.990326,2.349564,10.299103,-0.637360,-0.178589,0.644836
4,-7.967903,2.135628,10.735600,0.381583,0.223485,-0.701684
...,...,...,...,...,...,...
1004568,-6.134170,0.612335,7.551956,0.008247,-0.014966,0.014050
1004569,-4.242526,0.871489,8.839393,0.130020,-0.174484,-0.249451
1004570,-8.231445,-2.627792,8.759003,-0.959975,-1.172861,0.011912
1004571,-2.298405,1.225816,9.346847,-0.209832,0.058643,-0.080940


In [53]:
n_rows = X_train.shape[0]
n_cols = X_train.shape[1]
seg_shape = int(n_rows/400)
seg_shape = seg_shape * 400
seg_shape

1004400

In [54]:
X_train = X_train.head(seg_shape)
X_train

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z
0,-6.147430,1.798508,7.260559,0.852661,-0.562042,-0.006714
1,-5.669399,-0.153227,7.814577,-0.040317,0.056505,0.080634
2,-4.187195,-1.955337,7.799713,0.203247,0.073486,0.088348
3,-5.990326,2.349564,10.299103,-0.637360,-0.178589,0.644836
4,-7.967903,2.135628,10.735600,0.381583,0.223485,-0.701684
...,...,...,...,...,...,...
1004395,3.677448,-0.919362,6.129080,-0.215635,0.771523,0.854600
1004396,-4.640488,0.618286,14.400284,0.091568,0.958786,-0.319946
1004397,-5.647385,2.053650,6.504593,-0.378736,-0.291688,0.565356
1004398,6.760028,0.307655,8.748413,0.159454,-0.084595,-0.027008


In [55]:
segments = X_train.to_numpy()
segments.shape

(1004400, 6)

In [56]:
n_instances = int(segments.shape[0]/400)
n_attributes = segments.shape[1]
segments = segments.reshape(n_instances, 400, n_attributes)
np.save('data/X_train.npy', segments) 
segments.shape

(2511, 400, 6)

In [21]:
#labels = y_train.head(int((segments.shape[0])/400))
#labels.shape

In [22]:
labels = y_train.head(segments.shape[0]).head(2511)
labels.shape

(2511, 1)

In [23]:
# one hot encoding by pandas
label_ohe = pd.get_dummies(data=labels, columns=['label']).to_numpy()
label_ohe.shape

(2511, 6)

In [57]:
tf.keras.backend.clear_session()

input = Input(shape=(2400,))
reshape_input = Reshape((1, 400, 6))(input)

x = Conv2D(32, kernel_size = (1, 24), 
        strides = (1, 1),  
        activation = "relu", 
        padding = "valid", 
        kernel_regularizer = l2(1e-4))(reshape_input)
x = MaxPool2D((1, 4), (1, 2))(x)

x = Conv2D(64, kernel_size = (1, 16), 
        strides = (1, 1), 
        activation = "relu", 
        padding = "valid", 
        kernel_regularizer = l2(1e-4))(x)
x = MaxPool2D((1, 4), (1, 2))(x)

x = Conv2D(96, kernel_size = (1, 8), 
        strides = (1, 1), 
        activation = "relu", 
        padding = "valid", 
        kernel_regularizer = l2(1e-4))(x)
x = MaxPool2D((1, 4), (1, 2))(x)

x = Conv2D(128, kernel_size = (1, 4), 
        strides = (1, 1), 
        activation = "relu", 
        padding = "valid", 
        kernel_regularizer = l2(1e-4),
        name="encoder")(x)        
x = Flatten()(x)
output = Dense(6, activation = "softmax")(x)

model = Model(input, output)
model.compile(optimizer = Adam(3e-4), 
              loss = "categorical_crossentropy",
              metrics = ["categorical_accuracy"])

In [25]:
segments_reshaped = segments.reshape(-1, 2400)
segments_reshaped.shape

(2511, 2400)

In [26]:
model.fit(segments_reshaped, label_ohe, epochs = 20, 
          batch_size = 32, verbose = 2)

/home/ale/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4526: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/20
79/79 - 2s - loss: 1.8317 - categorical_accuracy: 0.1856 - 2s/epoch - 31ms/step
Epoch 2/20
79/79 - 2s - loss: 1.8013 - categorical_accuracy: 0.2015 - 2s/epoch - 29ms/step
Epoch 3/20
79/79 - 2s - loss: 1.7926 - categorical_accuracy: 0.2218 - 2s/epoch - 29ms/step
Epoch 4/20
79/79 - 2s - loss: 1.7817 - categorical_accuracy: 0.2234 - 2s/epoch - 29ms/step
Epoch 5/20
79/79 - 2s - loss: 1.7580 - categorical_accuracy: 0.2521 - 2s/epoch - 28ms/step
Epoch 6/20
79/79 - 2s - loss: 1.7160 - categorical_accuracy: 0.3007 - 2s/epoch - 31ms/step
Epoch 7/20
79/79 - 2s - loss: 1.6317 - categorical_accuracy: 0.3616 - 2s/epoch - 29ms/step
Epoch 8/20
79/79 - 2s - loss: 1.5015 - categorical_accuracy: 0.4524 - 2s/epoch - 29ms/step
Epoch 9/20
79/79 - 2s - loss: 1.3061 - categorical_accuracy: 0.5468 - 2s/epoch - 30ms/step
Epoch 10/20
79/79 - 2s - loss: 1.0292 - categorical_accuracy: 0.6854 - 2s/epoch - 30ms/step
Epoch 11/20
79/79 - 2s - loss: 0.7454 - categorical_accuracy: 0.7973 - 2s/epoch - 30ms/st

In [59]:
segments_reshaped = segments.reshape(-1, 2400)
segments_reshaped.shape

(2511, 2400)

In [61]:
labels.shape

(2511, 400, 6)

In [60]:
model.fit(segments, labels, epochs = 20, 
          batch_size = 32, verbose = 2)

Epoch 1/20


ValueError: Input 0 of layer "model" is incompatible with the layer: expected shape=(None, 2400), found shape=(32, 400, 6)

In [27]:
!rm -R par_model
!mkdir par_model

model = Model(model.input, model.get_layer("encoder").output)
tf.keras.models.save_model(model, "par_model", 
                           include_optimizer = False, 
                           save_format="tf")

2022-02-22 15:06:36.947733: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: par_model/assets


------------------

In [28]:
from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

In [29]:
base = bases.SavedModelBase("par_model")

head = Sequential([
    # Flatten(input_shape=(1, 35, 128)),
    Dense(units=128,
        activation='relu',
        kernel_regularizer=l2(0.0001),
        input_shape=(256,)),
    Dense(units=2,
        activation='softmax',
        kernel_regularizer=l2(0.0001)),
])

# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='adam')

In [30]:
!rm -R tflite_par_model
!mkdir tflite_par_model

converter = TFLiteTransferConverter(2,
              base,
              heads.KerasModelHead(head),
              optimizers.SGD(0.001),
              train_batch_size=5)

converter.convert_and_save('tflite_par_model')

/home/ale/Documents/Eldorado/HIACC/meta-3/ctxt_id/On-device-activity-recognition/tfltransfer/heads/keras_model_head.py:43: UserWarning: `tf.keras.experimental.export_saved_model` is deprecatedand will be removed in a future version. Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
  tf.compat.v1.keras.experimental.export_saved_model(keras_model, saved_model_dir)
/usr/lib/python3.8/contextlib.py:113: UserWarning: `tf.keras.backend.learning_phase_scope` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  return next(self.gen)


AttributeError: 'NoneType' object has no attribute 'op'

--------------------